# DataJoint U24 - Export Session

Same as before, import data.

In [1]:
import os
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')

In [ ]:
import datajoint as dj
dj.conn()
from element_lab import lab
from element_animal import subject
from element_session import session

Identify items for export with keys.

In [3]:
session_key=(session.Session&'subject="subject5"').fetch1('KEY')
mylab_key = (lab.Lab & 'lab="LabA"').fetch1('KEY')
myproj_key= (lab.Project & 'project="ProjA"').fetch1('KEY')
myprot_key= (lab.Protocol() & 'protocol="ProtA"').fetch1('KEY')

Get export function and related pynwb dependency, then export with keys from prev step.

In [4]:
from element_session.export import session_to_nwb_dict, session_to_nwb
help(session_to_nwb)

Help on function session_to_nwb in module element_session.export.nwb:

session_to_nwb(session_key)
    Generate one NWBFile object representing all session-level information,
      including session identifier, description, start time, etc.
    
    :param session_key: entry in session.Session table
    :return: NWBFile object



In [5]:
mynwbfile=session_to_nwb(session_key)

/Users/cb/miniconda3/envs/venv-nwb/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [6]:
from element_lab.export import elemlab_to_nwb_dict
help(elemlab_to_nwb_dict)

Help on function elemlab_to_nwb_dict in module element_lab.export.nwb:

elemlab_to_nwb_dict(lab_key=None, project_key=None, protocol_key=None)
    Generate a dictionary object containing all relevant lab information used when
        generating an NWB file at the session level. All parameters optional.
    Use: mynwbfile = NWBfile(identifier="your identifier",
                             session_description="your description",
                             session_start_time=session_datetime,
                             elemlab_to_nwb_dict(lab_key=key1,project_key=key2,protocol_key=key3))
    Note: The lab, project and protocol keys should specify one of their respective types.
    
    :param lab_key: Key specifying one entry in element_lab.lab.Lab
    :param project_key: Key specifying one entry in element_lab.lab.Project
    :param protocol_key: Key specifying one entry in element_lab.lab.PRotocol
    :return: dictionary with NWB parameters



In [7]:
from pynwb import NWBFile
lab_info = elemlab_to_nwb_dict(lab_key=mylab_key,project_key=myproj_key,protocol_key=myprot_key)
sess_info = session_to_nwb_dict(session_key)

In [8]:
mynwbfile = NWBFile(**sess_info,**lab_info)

/Users/cb/miniconda3/envs/venv-nwb/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


Learn more about using NWB formats [here](https://www.nwb.org/how-to-use/).

In [9]:
from pynwb import NWBHDF5IO
with NWBHDF5IO('session_metadata.nwb', mode='w') as io:
    io.write(mynwbfile)

/Users/cb/miniconda3/envs/venv-nwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:653: MissingRequiredBuildWarning: NWBFile 'root' is missing required value for attribute 'source_script_file_name'.
  warnings.warn(msg, MissingRequiredBuildWarning)


In [10]:
print(mynwbfile)

root pynwb.file.NWBFile at 0x140190306285216
Fields:
  experiment_description: Example project to populate element-lab
  file_create_date: [datetime.datetime(2021, 12, 6, 17, 1, 11, 974467, tzinfo=tzlocal())]
  identifier: subject5_20200415_111638
  institution: Example Uni
  keywords: ['Example' 'Study']
  lab: The Example Lab
  pharmacology: Subjects were administered 10ul sedative prior to surgery
  protocol: ProtA
  related_publications: ['arXiv:1807.11104' 'arXiv:1807.11104v1']
  session_description: Successful data collection, no notes
  session_start_time: 2020-04-15 11:16:38-05:00
  source_script: https://github.com/datajoint/element-lab/
  surgery: Craniotomy performed by session experimenter
  timestamps_reference_time: 2020-04-15 11:16:38-05:00

